# The model

In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# Define the model
class HandModel(nn.Module):
    def __init__(self):
        super(HandModel, self).__init__()
        self.fc1 = nn.Linear(63, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = nn.functional.softmax(x, dim=0)
        return x

model = HandModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Import the data

In [28]:
import os
import pandas as pd
def getalldatafile():
    res=[]
    for file in os.listdir("uploads"):
        file=os.path.join("uploads",file)
        print(file)
        if file.endswith(".txt") and file!="uploads\classes.txt":
            res.append(str(file.split(".")[0]))
    return res


df = pd.DataFrame(columns=['label'] + [f'd_{i:03}' for i in range(63)])
df.to_csv("uploads\data.csv", index=False,mode='w')

filenb=0
classes={}
for name in getalldatafile():
    print("file "+name+".txt")
    classes[name.removeprefix("uploads\\")]=filenb
    filenb+=1
    for line in open(name+".txt", "r").readlines():
        with open("uploads\data.csv", "a") as f:
            f.write(str(classes[name.removeprefix("uploads\\")])+","+line.removesuffix("\n")+"\n")
    print("file "+name+".txt done")

# ajout fichier de classe
with open("uploads\classes.txt", "w") as f:
    for key in classes.keys():
        f.write(key.removeprefix("uploads\\")+"\n")


uploads\a.txt
uploads\b.txt
uploads\c.txt
uploads\d.txt
uploads\data.csv
uploads\echec.txt
file uploads\a.txt
file uploads\a.txt done
file uploads\b.txt
file uploads\b.txt done
file uploads\c.txt
file uploads\c.txt done
file uploads\d.txt
file uploads\d.txt done
file uploads\echec.txt
file uploads\echec.txt done


# Init Dataset

In [29]:
# Load the data from the CSV file
data = pd.read_csv("uploads\data.csv")

# Separate the labels and the inputs
labels = data['label'].values
inputs = data.drop(columns=['label']).values

# Convert the inputs and labels to PyTorch tensors
inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Create a DataLoader
dataset = TensorDataset(inputs_tensor, labels_tensor)
train_ds,test_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
train_dl = DataLoader(train_ds, batch_size=32, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, shuffle=True)

# Evaluation

In [30]:
def evaluate_accuracy():
    iclasses = {v: k for k, v in classes.items()}
    model.eval()
    correct = 0
    total = 0
    class_correct = {class_name: 0 for class_name in classes}
    class_total = {class_name: 0 for class_name in classes}
    
    with torch.no_grad():
        for inputs, labels in test_dl:
            outputs = model(inputs)
            _,predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            for label, prediction in zip(labels, predicted):
                if label == prediction:
                    class_correct[iclasses[label.item()]] += 1
                class_total[iclasses[label.item()]] += 1
    
    global_accuracy = correct / total
    class_accuracy = {class_name: class_correct[class_name] / class_total[class_name] for class_name in classes}
    
    print(f'Global Accuracy: {global_accuracy:.2f}')
    for class_name, accuracy in class_accuracy.items():
        print(f'Accuracy of class {class_name}: {accuracy:.2f}')


evaluate_accuracy()

Global Accuracy: 0.17
Accuracy of class a: 0.21
Accuracy of class b: 0.46
Accuracy of class c: 0.03
Accuracy of class d: 0.00
Accuracy of class echec: 0.13



# TRAINING


In [31]:
evaluate_accuracy()
# Training loop
num_epochs = 400
for epoch in range(num_epochs):
    for inputs, labels in train_dl:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

evaluate_accuracy()

Global Accuracy: 0.18
Accuracy of class a: 0.20
Accuracy of class b: 0.47
Accuracy of class c: 0.01
Accuracy of class d: 0.00
Accuracy of class echec: 0.16
Epoch 1/400, Loss: 1.4628323316574097
Epoch 2/400, Loss: 1.4879530668258667
Epoch 3/400, Loss: 1.4596450328826904
Epoch 4/400, Loss: 1.4563589096069336
Epoch 5/400, Loss: 1.461188793182373
Epoch 6/400, Loss: 1.4595087766647339
Epoch 7/400, Loss: 1.4555799961090088
Epoch 8/400, Loss: 1.4581468105316162
Epoch 9/400, Loss: 1.4550267457962036
Epoch 10/400, Loss: 1.4560660123825073
Epoch 11/400, Loss: 1.4559375047683716
Epoch 12/400, Loss: 1.4543983936309814
Epoch 13/400, Loss: 1.455106496810913
Epoch 14/400, Loss: 1.454464077949524
Epoch 15/400, Loss: 1.4548965692520142
Epoch 16/400, Loss: 1.4579129219055176
Epoch 17/400, Loss: 1.4556982517242432
Epoch 18/400, Loss: 1.453982949256897
Epoch 19/400, Loss: 1.4542109966278076
Epoch 20/400, Loss: 1.4546654224395752
Epoch 21/400, Loss: 1.4546535015106201
Epoch 22/400, Loss: 1.4545294046401978

# Save Model

In [32]:
#Save the model
torch.save(model.state_dict(), 'uploads\handmodel.pth')